In [34]:
import torchvision
from research_task_utils import *
import importlib
from torchsummary import summary
import numpy as np
# importlib.reload(research_task_utils)

In [17]:
batchsize = 512
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor()
    ]
)
traindata = torchvision.datasets.CIFAR100(root="./dataset", train=True, download=True, transform=transform)
testdata = torchvision.datasets.CIFAR100(root="./dataset", train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=batchsize, shuffle=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=batchsize, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
from collections import OrderedDict
class ConvBlock(torch.nn.Module):
    def __init__(self, num_of_convs, c_in, c_out, wh):
        super().__init__()
        
        self.num_of_convs = num_of_convs
        self.c_in = c_in
        self.c_out = c_out
        self.wh = wh
        
        layers = []
        
        assert num_of_convs > 0
        layers.append(('conv1',torch.nn.Conv2d(c_in, c_out, kernel_size=3, stride=1, padding=1)))
        for k in range(1, num_of_convs):
            layers.append((f'relu_{k}',torch.nn.ReLU()))
            layers.append((f'conv_{k+1}',torch.nn.Conv2d(c_out, c_out, kernel_size=3, stride=1, padding=1)))
            
        layers.append(('bn',torch.nn.BatchNorm2d(c_out)))
        layers.append((f'relu_{num_of_convs}',torch.nn.ReLU()))
        
        self.layers = torch.nn.Sequential(OrderedDict(layers))
        
    def num_of_params(self, only_trainable: bool = False):
        parameters = list(self.parameters())
        if only_trainable:
            parameters = [p for p in parameters if p.requires_grad]
        unique = {p.data_ptr(): p for p in parameters}.values()
        return sum(p.numel() for p in unique)
        
    def forward(self, x):
        return self.layers(x)
    
def num_of_convBlock_params(num_of_convs, c_in, c_out, wh):
    num = c_in*c_out*9 + c_out # first conv layer
    num += (num_of_convs - 1)*(c_out*c_out*9 + c_out) # other conv layers
    num += 2*c_out # BN params
    return num

In [19]:
class architecture2():
    def __init__(self, c1, c2, c3, c4, k1, k2, k3, k4):
        self.c1 = c1
        self.c2 = c2
        self.c3 = c3
        self.c4 = c4
        self.k1 = k1
        self.k2 = k2
        self.k3 = k3
        self.k4 = k4
        
        self.num_of_params = (num_of_convBlock_params(num_of_convs=k1, c_in=3, c_out=c1, wh=32) +
                              num_of_convBlock_params(num_of_convs=k2, c_in=c1, c_out=c2, wh=16) +
                              num_of_convBlock_params(num_of_convs=k3, c_in=c2, c_out=c3, wh=8) +
                              num_of_convBlock_params(num_of_convs=k4, c_in=c3, c_out=c4, wh=4) +
                              c4*100+100)
        
    def get_num_of_params(self):
        return self.num_of_params

In [30]:
class ConvArchitecture2(torch.nn.Module):
    def __init__(self, model_hyperparams):
        super().__init__()
        
        c1, c2, c3, c4, k1, k2, k3, k4 = model_hyperparams
        self.hyperparams = model_hyperparams
        
        self.layers = torch.nn.Sequential(
            ConvBlock(num_of_convs=k1, c_in=3, c_out=c1, wh=32),
            torch.nn.MaxPool2d(kernel_size=2),
            
            ConvBlock(num_of_convs=k2, c_in=c1, c_out=c2, wh=16),
            torch.nn.MaxPool2d(kernel_size=2),
            
            ConvBlock(num_of_convs=k3, c_in=c2, c_out=c3, wh=8),
            torch.nn.MaxPool2d(kernel_size=2),
            
            ConvBlock(num_of_convs=k4, c_in=c3, c_out=c4, wh=4),
            
            torch.nn.AvgPool2d(kernel_size=4),
            torch.nn.Flatten(),
            torch.nn.Linear(c4, 100),
            torch.nn.Softmax(dim=-1)
        )
    
    def num_of_params(self, only_trainable: bool = False):
        parameters = list(self.parameters())
        if only_trainable:
            parameters = [p for p in parameters if p.requires_grad]
        unique = {p.data_ptr(): p for p in parameters}.values()
        return sum(p.numel() for p in unique)

    def forward(self, x):
        return self.layers(x)

In [32]:
import statistics
import json
import os

def add_statistics_to_json(model_info, statistics_dict, path):
    if not os.path.exists(path):
        with open(path, 'w') as f:
            json.dump([{"model_info": model_info, "statistics": statistics_dict}], f)
    else:
        with open(path) as f:
            old_data = json.load(f)
        with open(path, 'w') as f:
            json.dump(old_data+[{"model_info": model_info, "statistics": statistics_dict}], f)
    

def perform_experiment(device, trainloader, testloader, Model_class, model_hyperparams, epochs_n, repeats_n, path):
    train_accs = []
    test_accs = []
    
    for k in range(repeats_n):
        model = Model_class(model_hyperparams).to(device)
        optimizer = torch.optim.Adam(model.parameters())
        loss_function = torch.nn.CrossEntropyLoss()
        statistics_list = init_statistics(model, device, trainloader, testloader, loss_function)
        train(model, device, optimizer, loss_function, trainloader, testloader, statistics_list, epochs_n=epochs_n)
        train_accs.append(statistics_list["trainacc"][-1])
        test_accs.append(statistics_list["testacc"][-1])
        
        print("=================")
        print("Architecture: ", model_hyperparams)
        print("test acc:", statistics_list['testacc'][-1])
        plot_statistics(statistics_list)
        add_statistics_to_json(model_info={"name": "architecture2", "configuration": model_hyperparams,
                                           "comment": "third gridsearch with grads, 100 epochs"},
                               statistics_dict=statistics_list,path=path)
        print("=================")
        
    if repeats_n != 1:
        test_accs_std = statistics.stdev(test_accs)
        train_accs_std = statistics.stdev(train_accs)
    else:
        test_accs_std, train_accs_std = 0, 0
        
    result = {"train_acc_mean": statistics.mean(train_accs),
              "train_acc_std": train_accs_std,
              "test_acc_mean": statistics.mean(test_accs),
              "test_acc_std": test_accs_std,
              "num_of_params": model.num_of_params()
             }
    if repeats_n == 1:
        result["statistics"] = statistics_list
    return result


def perform_experiments(device, trainloader, testloader, Model_class, hyperparams_list, epochs_n, repeats_n, path):
    results = dict()
    for hyperparam in hyperparams_list:
        results[hyperparam] = perform_experiment(device, trainloader, testloader,
                                                   Model_class, hyperparam, epochs_n, repeats_n, path)
    return results

    
def get_N(hyperparams, data_to_compute_gradients):
    return data_to_compute_gradients[hyperparams][0]

def get_acc(hyperparams, data_to_compute_gradients):
    return data_to_compute_gradients[hyperparams][1]

def get_grad_N(hyperparams, data_to_compute_gradients):
    grad = []
    for i in range(len(hyperparams)):
        hyperparams_tmp = list(hyperparams)
        hyperparams_tmp[i] += 1
        grad.append(get_N(tuple(hyperparams_tmp), data_to_compute_gradients) 
                    - get_N(hyperparams, data_to_compute_gradients))
    return np.array(grad)

def get_grad_acc(hyperparams, data_to_compute_gradients):
    grad = []
    for i in range(len(hyperparams)):
        hyperparams_tmp = list(hyperparams)
        hyperparams_tmp[i] += 1
        grad.append(get_acc(tuple(hyperparams_tmp), data_to_compute_gradients) 
                    - get_acc(hyperparams, data_to_compute_gradients))
    return np.array(grad)

def generate_architectures_to_check(initial_hyperparams):
    architectures = [initial_hyperparams]
    for i in range(len(initial_hyperparams)):
        hyperparams_tmp = list(initial_hyperparams)
        hyperparams_tmp[i] += 1
        architectures.append(tuple(hyperparams_tmp))
    return architectures

In [84]:
# # замерджить файлы с колаба и с пк

# with open("models_logs_gradsearch.json") as f:
#     data_pc = json.load(f)
# with open("models_logs_gradsearch_colab.json") as f:
#     data_colab = json.load(f)
# with open("models_logs_gradsearch_merged.json", 'w') as f:
#     json.dump(data_pc + list(reversed(data_colab)), f)

In [ ]:
initial_hyperparams = (8, 8, 16, 32, 4, 4, 4, 4) #c1, c2, c3, c4, k1, k2, k3, k4
path = "models_logs_gradsearch3_test.json"
K = 5
while (True):
    architectures = generate_architectures_to_check(initial_hyperparams)
    results = perform_experiments(device, trainloader, testloader,
                                  ConvArchitecture2, architectures, epochs_n=1, repeats_n=1, path=path)
    
    
    with open(path) as f:
        logs = json.load(f)
        
    data_to_compute_gradients = {}
    for log in logs:
        configuration = log["model_info"]["configuration"]
        c1, c2, c3, c4, k1, k2, k3, k4 = tuple(configuration)
        num_of_params = architecture2(c1, c2, c3, c4, k1, k2, k3, k4).get_num_of_params()
        acc_avg = statistics.mean(log["statistics"]["testacc"][-10:])
        data_to_compute_gradients[tuple(configuration)] =  (num_of_params, acc_avg)
        
    grad_N = get_grad_N(initial_hyperparams, data_to_compute_gradients)
    grad_acc = get_grad_acc(initial_hyperparams, data_to_compute_gradients)

    direction_to_change_params = grad_acc - np.dot(grad_N, grad_acc) / np.dot(grad_N, grad_N) * grad_N
    direction_to_change_params /= np.linalg.norm(direction_to_change_params)
        
    new_hyperparams = np.array(initial_hyperparams) + K*direction_to_change_params
    new_hyperparams = np.rint(new_hyperparams).astype(int)
    
    for i in range(8):
        if new_hyperparams[i] < 1:
            new_hyperparams[i] = 1
    
    initial_hyperparams = tuple(new_hyperparams)
    print("NEW INITIAL PARAMS: ", initial_hyperparams)

  0%|                                                                                         | 0/1 [00:00<?, ?it/s]